In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict, Annotated, Literal
from pydantic import BaseModel, Field
import operator


D:\LangChain\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",temperature=0)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [3]:
class Sentiment(BaseModel):
    sentiment: Literal["positive", "negative"] = Field(description="Sentiment of the review")

In [ ]:
class Diagnosis(BaseModel):
    issue_type: Literal["UX","Bug","Support","Other"] = Field(description="Type of issue")
    tone: Literal["angry","frustrated","disappointed","calm"] = Field(description="Tone of the review")
    urgency: Literal["low","medium","high"] = Field(description="How urgent or critical is the issue appears to be")

In [4]:
structured_llm = llm.with_structured_output(Sentiment)
structured_llm2 = llm.with_structured_output(Diagnosis)

In [7]:
class ReviewState(TypedDict):
    review: str
    sentiment: Literal["positive", "negative"]
    diagnosis: dict
    response: str

In [8]:
def find_sentiment(state: ReviewState):
    prompt = f'for the following review: "{state["review"]}" what is the sentiment?'
    sentimenet = structured_llm.invoke(prompt)
    return {"sentiment": sentimenet.sentiment}

def check_sentiment(state: ReviewState) -> Literal["positive_feedback", "run_diagnosis"]:
    if state['sentiment'] == 'positive':
        return 'positive_feedback'
    else:
        return 'run_diagnosis'

def positive_response(state: ReviewState):
    prompt = f"""Write a warm thank-you message in response to the review: \n\n\ "{state['review']}\"\n
    Also, kindly ask the user to leave a feedback on our website.

    """
    response = llm.invoke(prompt)
    return {"response": response}

def run_diagnosis(state: ReviewState):
    prompt = f"""Diagnose this negative review:\n\n{state['review']}\n
    Return issue_type, tone, urgency.

"""
    response = structured_llm2.invoke(prompt)
    return {"diagnosis": response.model_dump()}

def negative_response(state: ReviewState):
    diagnosis = state["diagnosis"]
    prompt = f"""You are a support assistant.
    The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgentcy as '{diagnosis['urgency']}'. Write an empathetic, helpful response to the user.

"""
    response = llm.invoke(prompt).content
    return {'response':response}



In [ ]:
graph = StateGraph(ReviewState)

graph.add_node('find_sentiment',find_sentiment)
graph.add_node('run_diagnosis',run_diagnosis)
graph.add_node('negative_response',negative_response)

graph.add_edge(START, 'find_sentiment')
graph.add_conditional_edges('find_sentiment',check_sentiment)
graph.add_edge('run_diagnosis', 'negative_response')
graph.add_edge('negative_response', END)
graph.add_edge('positive_response',END)
graph.add_edge('find_sentiment',END)

workflow = graph.compile()
workflow


In [5]:
prompt = "You guys suck. No great products"
structured_llm.invoke(prompt)

ChatGoogleGenerativeAIError: Error calling model 'gemini-2.0-flash' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash\nPlease retry in 10.302291255s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quo